In [1]:
import sys
sys.path.append("..")

In [2]:
import default_config as cfg
import remote_storage_config as storage_cfg

cfg.__dict__.update(storage_cfg.__dict__)

In [3]:
import dataset_functions
import models.six_stage_linear_model as model
import dataset_builder
import utils
import tensorflow as tf

In [4]:
model_ds=model.ModelDatasetComponent(cfg)

tfrecord_files_train=dataset_builder.get_tfrecord_filenames(cfg.TRAIN_TFRECORDS,cfg)
tfrecord_files_valid=dataset_builder.get_tfrecord_filenames(cfg.VALID_TFRECORDS,cfg)
print("Found the following training TFrecords:\n","\n".join(tfrecord_files_train))
print("Found the following validation TFrecords:\n","\n".join(tfrecord_files_valid))

print("Building training dataset")

def build_training_ds(tfrecord_filenames: list, labels_placement_function, config) -> tf.data.Dataset:
    """    :param config: effective config dict
    :param labels_placement_function: a model function, which applies the last stage of transformation to the dataset, to distribute the
    various labels correctly according to model outputs
    :param tfrecord_filenames: should be list of correct TFrecord filename, either local or remote (gcs, with gs:// prefix)"""
    # TFrecord files to raw format
    dataset_transformer = dataset_functions.DatasetTransformer(config)
    ds = tf.data.TFRecordDataset(tfrecord_filenames)  # numf reads can be put here, but I don't think I/O is the bottleneck

    # raw format to imgs,tensors(coords kpts)
    ds = ds.map(dataset_transformer.read_tfrecord)

    # cache  ,caching is here before decompressing jpgs and label tensors (should be ~9GB) , (full dataset should be ~90, cache later if RAM aviable)
    if config.CACHE: ds = ds.cache()
    if config.SHUFFLE: ds = ds.shuffle(100)

    ds = ds.map(dataset_transformer.open_image) #jpeg to array
    # Augmentation should be here, to operate on smaller tensors

    
    ds = ds.map(dataset_transformer.make_label_tensors) # tensors to label_tensors (46,46,17/38)
    ds = ds.map(labels_placement_function) # imgs,label_tensors arrange for model outputs

    ds = ds.batch(config.BATCH_SIZE)
    ds = ds.repeat()
    if config.PREFETCH: ds = ds.prefetch(config.PREFETCH)
    return ds


dst=build_training_ds(tfrecord_files_train,model_ds.place_training_labels,cfg)


Retrieving TFrecords from: gs://datasets_bucket_a/training
Retrieving TFrecords from: gs://datasets_bucket_a/validation
Found the following training TFrecords:
 gs://datasets_bucket_a/training-001.tfrecords
gs://datasets_bucket_a/training-002.tfrecords
gs://datasets_bucket_a/training-003.tfrecords
gs://datasets_bucket_a/training-004.tfrecords
gs://datasets_bucket_a/training-005.tfrecords
gs://datasets_bucket_a/training-006.tfrecords
gs://datasets_bucket_a/training-007.tfrecords
gs://datasets_bucket_a/training-008.tfrecords
gs://datasets_bucket_a/training-009.tfrecords
gs://datasets_bucket_a/training-010.tfrecords
gs://datasets_bucket_a/training-011.tfrecords
gs://datasets_bucket_a/training-012.tfrecords
gs://datasets_bucket_a/training-013.tfrecords
gs://datasets_bucket_a/training-014.tfrecords
gs://datasets_bucket_a/training-015.tfrecords
gs://datasets_bucket_a/training-016.tfrecords
gs://datasets_bucket_a/training-017.tfrecords
gs://datasets_bucket_a/training-018.tfrecords
gs://datase